# Cummulative Industry Exposure

In [1]:
# Import libraries

# System

import os

# Analysis

import numpy as np # Numerical Analysis
import pandas as pd # Data Mgmt
import geopandas as gpd # Spatial Data Mgmt
import rasterio # Rasters
from rasterio.transform import Affine
from rasterio import features

# Visualization

import matplotlib.pyplot as plt # Basic Plotting
plt.style.use('ggplot') # 
import seaborn as sns # Statistical Plotting
import contextily # Base Map Visualization

In [2]:
# Definitions

def Save_array_to_geotiff_template(array, template_path, save_path):
    '''Saves a numpy array into a geotiff with the same CRS as the template.
    '''            

    # Get metadata from template
    rst = rasterio.open(template_path) # Open template
    meta = rst.meta.copy() # Copy template metadata
    # meta.update(compress='lzw') # Good for integers/categorical rasters
    rst.close()

    with rasterio.open(save_path, 'w+', **meta) as out: # Burn features into raster
        out.write_band(1, array)

In [3]:
rst = rasterio.open('../2_Model_Pollutant_Exposure/template.tif') # Open template
meta = rst.meta.copy() # Copy template metadata
# meta.update(compress='lzw') # Good for integers/categorical rasters
rst.close()

In [4]:
# Load raster template

raster_path = os.path.join(os.getcwd(), '..', '2_Model_Pollutant_Exposure', 'template.npy')

raster = np.load(raster_path)

# Minneapolis Boundary

mpls_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'mpls_boundary.geojson')
mpls = gpd.read_file(mpls_path)

In [5]:
# Load rasterized exposures

# Filenames

path_to_layers = os.path.join(os.getcwd(), '..', '2_Model_Pollutant_Exposure', 'Modeled_Exposure')

filenames = os.listdir(path_to_layers)
filenames.remove('.ipynb_checkpoints')
layernames = []

rasts = []

for filename in filenames:

    path = os.path.join(path_to_layers, filename)
    
    layername = filename.split('_')[0]
    
    if layername != 'AADT' and layername != '.ipynb_checkpoints': # Current or cummulative for industrial pollutants
        layername += '_' + filename.split('_')[1]
        
    layernames += [layername]
    
    rasterio_rast = rasterio.open(path)
    
    rasts += [rasterio_rast.read(1)]
    
    rasterio_rast.close()

In [6]:
layernames

['PM10Primary_cummulative',
 'PMPrimary_cummulative',
 'PMPrimary_current',
 'PM25Primary_current',
 'PM25Primary_cummulative',
 'SulfurDioxide_current',
 'VolatileOrganicCompounds_current',
 'AADT',
 'PM10Primary_current',
 'VolatileOrganicCompounds_cummulative',
 'SulfurDioxide_cummulative']

In [7]:
# Weights? Let's try just looking at current pm2.5 and AADT
# Weighted equally

weights_tests = {}

# First let's look at current pm2.5 and aadt

for percent in np.linspace(0, 1, num=11):
    
    testname = str(int(percent*100)) + 'PM-' + str(int((1-percent)*100)) + 'T'

    weights_tests[testname] = np.array([0,0,0,percent,0,0,0,1-percent,0,0,0])

In [8]:
# Sum them up?

tests = {}

for weights_testname in weights_tests:
    
    weights = weights_tests[weights_testname]

    sums = np.zeros(rasts[0].shape)

    for i, rast in enumerate(rasts):

        sums += rast*weights[i]
        
    tests[weights_testname] = sums

## Save

In [16]:
for testname in tests:
    array = tests[testname]
    template_path = '../2_Model_Pollutant_Exposure/template.tif'
    save_path = os.path.join('Tests', testname + '_test.tif')
    Save_array_to_geotiff_template(array, template_path, save_path)